##  Data Preparation

In [6]:
# TODOs
# _ _ _ _ _

# 250000 words

# El potter no [MASK] novia

# p _ t t e r


# 1. Filtrar las palabras
# 2. Palabras mas comunes
# 3. Data colador
# 4. Penalize more small words??
# 5. Podria concatenar las palabras adivinadas hasta ahora al final del classifier, es practicamente concatenar un one-hot vector 
# de 0-26 con 1 donde las palabras ya fueron adivinadas.
# 6. Podria usar one-hot encodings 0-27 para los characteres
# 7. El Test y el val tambien generalo de manera dinamica. No te preocupes por que distribucion tendra.


In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import DatasetDict, Dataset


In [36]:
# Load datasets 
df_train = pd.read_csv('data/to_mask_train.csv', converters={'word': str})
df_val = pd.read_csv('data/masked_val.csv', converters={'masked_word': str, 'labels': str, 'previous_guesses': eval})
df_test = pd.read_csv('data/masked_test.csv', converters={'masked_word': str, 'labels': str, 'previous_guesses': eval})

print(df_train.head())

# Add spaces to the word in training
df_train["word"] = df_train["word"].apply(lambda x: ' '.join(list(x)))

# Add spaces to the masked_word and labels
df_val["masked_word"] = df_val["masked_word"].apply(lambda x: ' '.join(list(x)))
df_val["labels"] = df_val["labels"].apply(lambda x: ' '.join(list(x)))
df_test["masked_word"] = df_test["masked_word"].apply(lambda x: ' '.join(list(x)))
df_test["labels"] = df_test["labels"].apply(lambda x: ' '.join(list(x)))

# Print the shapes of the datasets
print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

          word  word_length  unique_chars
0      timpani            7             6
1       worsle            6             6
2        yinst            5             5
3  grangerized           11             8
4      matatua            7             4
(204570, 3)
(22730, 3)
(130436, 3)


In [42]:
df_train.head()

,word,word_length,unique_chars
0,t i m p a n i,7,6
1,w o r s l e,6,6
2,y i n s t,5,5
3,g r a n g e r i z e d,11,8
4,m a t a t u a,7,4


In [43]:
df_val.head()

,masked_word,labels,previous_guesses
0,h e _ _ c l _ t _ s,h e r a c l i t u s,"[0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, ..."
1,_ e _ o d _ _ _,m e r o d a c h,"[0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
2,_ _ g _ _ _,i n g i r t,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,_ _ _ z u o k a,s h i z u o k a,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, ..."
4,m _ _ t _ _ _ _ n n e _ _ e d,m u l t i c h a n n e l l e d,"[0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, ..."


In [44]:
# Gather all the data in a DatasetDict
df_train.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)
df_val.reset_index(drop=True, inplace=True)

data = DatasetDict({
    "train": Dataset.from_pandas(df_train[["word"]]),
    "valid": Dataset.from_pandas(df_val),
    "test": Dataset.from_pandas(df_test)
})

data

DatasetDict({
    train: Dataset({
        features: ['word'],
        num_rows: 204570
    })
    valid: Dataset({
        features: ['masked_word', 'labels', 'previous_guesses'],
        num_rows: 22730
    })
    test: Dataset({
        features: ['masked_word', 'labels', 'previous_guesses'],
        num_rows: 130436
    })
})

In [129]:
import random
import torch
from transformers import DataCollatorForLanguageModeling, BertTokenizer
from torch.utils.data import DataLoader

class CustomDataCollatorForMLM(DataCollatorForLanguageModeling):
    def __init__(self, tokenizer, mlm_probability=0.75, max_length=42):
        super().__init__(tokenizer=tokenizer, mlm_probability=mlm_probability)
        self.max_length = max_length
        self.char_to_index = {chr(i + ord('a')): i for i in range(26)}

    
    def __call__(self, examples):

        # Convert characters to their corresponding numerical labels
        labels = []
        words = []
        for example in examples:
            example = example['word']
            row_label = [self.char_to_index[char] for char in example.split()]
            labels.append(row_label)
            words.append(example)

        # Pad labels to the same length (considering special tokens at the beginning and end of each label)
        labels = [ [-100] + row_label + [-100] * (self.max_length - len(row_label) - 1) for row_label in labels]
        labels = torch.tensor(labels)

        # Create a one-hot vector for the labels without consider the -100 values
        prev_guess = torch.zeros((labels.shape[0], 26))
        for i, label in enumerate(labels):
            prev_guess[i][label[label != -100]] = 1

        # Tokenize and pad the input examples
        batch = self.tokenizer(words, truncation=True, padding='max_length', return_tensors="pt", max_length=self.max_length)
        
        # Get the input_ids and apply masking
        input_ids = batch["input_ids"]
        # labels = input_ids.clone()

        for i, input_id in enumerate(input_ids):

            # Get unique tokens
            unique_tokens = torch.unique(input_id)
            
            # Filter Special Tokens by setting probability to 0.0
            special_tokens_mask = self.tokenizer.get_special_tokens_mask(unique_tokens, already_has_special_tokens=True)
            probabilities = torch.full(unique_tokens.shape, self.mlm_probability)
            probabilities.masked_fill_(torch.tensor(special_tokens_mask, dtype=torch.bool), value=0.0)
            
            # Filter the tokens to mask
            tokens_to_mask = unique_tokens[torch.bernoulli(probabilities).bool()]

            # Mask all instances of the chosen tokens
            masked_indices = torch.zeros(input_id.shape, dtype=torch.bool)
            for token in tokens_to_mask:
                masked_indices[(input_id == token)] = True

            labels[i][~masked_indices] = -100  # We only compute loss on masked tokens

            # 80% of the time, replace masked input tokens with tokenizer.mask_token ([MASK])
            indices_replaced = torch.bernoulli(torch.full(input_id.shape, 0.8)).bool() & masked_indices
            input_ids[i][indices_replaced] = self.tokenizer.convert_tokens_to_ids(self.tokenizer.mask_token)

            # 10% of the time, replace masked input tokens with random word
            # indices_random = torch.bernoulli(torch.full(input_id.shape, 0.5)).bool() & masked_indices & ~indices_replaced
            # random_words = torch.randint(len(self.tokenizer), input_id.shape, dtype=torch.long)
            # input_ids[i][indices_random] = random_words[indices_random]

            # The rest of the time (10% of the time) we keep the masked input tokens unchanged

        # Set to zero the values masked in the labels
        for i, label in enumerate(labels):
            prev_guess[i][label[label != -100]] = 0

        batch["input_ids"] = input_ids
        batch["labels"] = labels
        batch["prev_guess"] = prev_guess
        return batch


In [132]:
# TOY EXAMPLE
# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Initialize the custom data collator
data_collator = CustomDataCollatorForMLM(tokenizer, mlm_probability=0.5, max_length=10)

# Toy dataset
toy_data = {
    "word": ["h e l l o", "w o r l d", "p a r i s", "b e r l i n"]
}
df_temp = pd.DataFrame(toy_data)
print(df_temp)

# Tokenize the toy dataset
TensorDataset = Dataset.from_pandas(df_temp)
print(TensorDataset)
toy_dataloader = DataLoader(TensorDataset, batch_size=2, collate_fn=data_collator)

# Print the first batch
for batch in toy_dataloader:
    print(batch['input_ids'])
    print(batch['labels'])
    print(batch['prev_guess'])
    break

          word
0    h e l l o
1    w o r l d
2    p a r i s
3  b e r l i n
Dataset({
    features: ['word'],
    num_rows: 4
})
tensor([[ 101, 1044, 1041,  103, 1048, 1051,  102,    0,    0,    0],
        [ 101, 1059, 1051, 1054,  103, 1040,  102,    0,    0,    0]])
tensor([[-100, -100, -100,   11,   11, -100, -100, -100, -100, -100],
        [-100, -100, -100, -100,   11, -100, -100, -100, -100, -100]])
tensor([[0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1.,
         0., 0., 0., 0., 1., 0., 0., 0.]])


In [131]:
# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Initialize the custom data collator
data_collator = CustomDataCollatorForMLM(tokenizer, mlm_probability=0.5, max_length=42)

train_dataloader = DataLoader(
    data['train'], 
    batch_size=64,
    collate_fn = data_collator
)

# Prepare a batch using the custom data collator
for batch in train_dataloader:

    print("input_ids", batch['input_ids'][:10,:10], batch['input_ids'].shape)
    print("labels", batch['labels'][:10,:10], batch['labels'].shape)
    print("prev_guess", batch['prev_guess'][:10,:10], batch['prev_guess'].shape)
    break



input_ids tensor([[ 101,  103, 1045, 1049, 1052, 1037, 1050, 1045,  102,    0],
        [ 101, 1059,  103,  103, 1055,  103,  103,  102,    0,    0],
        [ 101, 1061, 1045,  103, 1055, 1056,  102,    0,    0,    0],
        [ 101,  103, 1054,  103, 1050,  103, 1041, 1054,  103, 1062],
        [ 101, 1049,  103, 1056,  103, 1056, 1057, 1037,  102,    0],
        [ 101,  103, 1045, 1055, 1056,  103, 1054, 1045, 1041, 1056],
        [ 101,  103,  103, 1039, 1051, 1048, 1048,  103, 1039, 1056],
        [ 101, 1039,  103,  103, 1061,  103,  103, 1041, 1040, 1045],
        [ 101,  103,  103, 1045, 1050,  103,  102,    0,    0,    0],
        [ 101,  103, 1041, 1054, 1045,  103,  103, 1039, 1037, 1054]]) torch.Size([64, 42])
labels tensor([[-100,   19, -100, -100, -100, -100, -100, -100, -100, -100],
        [-100, -100,   14,   17, -100,   11,    4, -100, -100, -100],
        [-100, -100, -100,   13, -100, -100, -100, -100, -100, -100],
        [-100,    6, -100,    0, -100,    6, -100, 

# Desing the model for fine tuning

In [133]:
from datasets import load_dataset,Dataset,DatasetDict
from transformers import DataCollatorWithPadding,AutoModelForSequenceClassification, Trainer, TrainingArguments,AutoTokenizer,AutoModel,AutoConfig
from transformers.modeling_outputs import TokenClassifierOutput
import torch
import torch.nn as nn
import pandas as pd

In [138]:
class HangmanNet(nn.Module):
  def __init__(self,checkpoint, vocab_size = 26, unfreeze_layers = 0): 
    super(HangmanNet,self).__init__() 
    self.num_labels = vocab_size 

    #Load Model with given checkpoint and extract its body
    self.model = AutoModel.from_pretrained(checkpoint,config=AutoConfig.from_pretrained(checkpoint, output_attentions=True,output_hidden_states=True))
    
    # Freeze all layers in the BERT model
    for param in self.model.parameters():
        param.requires_grad = False

    # Unfreeze the last `unfreeze_layers` layers
    if unfreeze_layers > 0:
        for layer in self.model.encoder.layer[-unfreeze_layers:]:
            for param in layer.parameters():
                param.requires_grad = True
        

    self.dropout = nn.Dropout(0.1) 
    self.classifier = nn.Linear(768,vocab_size) # load and initialize weights
  

  def forward(self, input_ids=None, attention_mask=None, labels=None):
      outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
      
      sequence_output = outputs.last_hidden_state  # (batch_size, sequence_length, hidden_size)
      sequence_output = self.dropout(sequence_output)
      logits = self.classifier(sequence_output)  # (batch_size, sequence_length, num_labels)

      loss = None
      if labels is not None:
          loss_fct = nn.CrossEntropyLoss()

          # NOTE: I already has the labels in active logits representation

          # Only keep active parts of the loss
        #   active_loss = labels.view(-1) != -100
          active_logits = logits.view(-1, self.num_labels)
        #   active_labels = torch.where(active_loss, labels.view(-1), torch.tensor(loss_fct.ignore_index).type_as(labels))
          loss = loss_fct(active_logits, labels.view(-1))
        
      return TokenClassifierOutput(logits=logits, loss=loss, hidden_states=outputs.hidden_states,attentions=outputs.attentions)


In [140]:
# Check the model output
device = "cpu"
checkpoint = "bert-base-uncased"
model = HangmanNet(checkpoint=checkpoint, vocab_size = 26, unfreeze_layers = 1).to(device)

# Prepare a batch using the custom data collator
for batch in train_dataloader:

    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = batch['labels']

    # Forward pass
    outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
    print(outputs.loss)
    print(outputs.logits.shape)

    break


/home/zosov/anaconda3/envs/character-bert/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tensor(3.2538, grad_fn=<NllLossBackward0>)
torch.Size([64, 42, 26])


In [141]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HangmanNet(checkpoint=checkpoint, vocab_size = 26, unfreeze_layers = 1).to(device)

# Print the trainable parameters of the model
print("Trainable parameters:")
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)  

Trainable parameters:
model.encoder.layer.11.attention.self.query.weight
model.encoder.layer.11.attention.self.query.bias
model.encoder.layer.11.attention.self.key.weight
model.encoder.layer.11.attention.self.key.bias
model.encoder.layer.11.attention.self.value.weight
model.encoder.layer.11.attention.self.value.bias
model.encoder.layer.11.attention.output.dense.weight
model.encoder.layer.11.attention.output.dense.bias
model.encoder.layer.11.attention.output.LayerNorm.weight
model.encoder.layer.11.attention.output.LayerNorm.bias
model.encoder.layer.11.intermediate.dense.weight
model.encoder.layer.11.intermediate.dense.bias
model.encoder.layer.11.output.dense.weight
model.encoder.layer.11.output.dense.bias
model.encoder.layer.11.output.LayerNorm.weight
model.encoder.layer.11.output.LayerNorm.bias
classifier.weight
classifier.bias


In [ ]:
from transformers import AdamW, get_scheduler

optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

11721


/home/zosov/anaconda3/envs/character-bert/lib/python3.10/site-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from datasets import load_metric
metric = load_metric("f1")
metric_acc = load_metric("accuracy")

/home/zosov/anaconda3/envs/character-bert/lib/python3.10/site-packages/datasets/load.py:759: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/home/zosov/anaconda3/envs/character-bert/lib/python3.10/site-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=

In [ ]:
device

device(type='cuda')

In [ ]:
train_dataloader = DataLoader(
    data["train"], 
    batch_size=64,
    num_workers=4,
    collate_fn=data_collator,
    pin_memory=True)
    
    # collate_fn=data_collator)
eval_dataloader = DataLoader(data["valid"],
                             batch_size=64,
                             num_workers=8,
                             pin_memory=True)
                             # collate_fn=data_collator)

In [ ]:
from torch.utils.tensorboard import SummaryWriter
import torch
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score, f1_score
import os

# Initialize TensorBoard writer
writer = SummaryWriter(log_dir='tensorboard_logs')

# Initialize metrics storage
train_losses = []
val_losses = []
val_accuracies = []
val_f1_scores = []

for epoch in range(num_epochs):
    model.train()
    total_train_loss = 0

    for batch in tqdm(train_dataloader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_dataloader)
    train_losses.append(avg_train_loss)

    # Validation
    model.eval()
    total_val_loss = 0
    all_predictions = []
    all_references = []

    for batch in tqdm(eval_dataloader, desc=f"Validation Epoch {epoch+1}/{num_epochs}"):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        loss = outputs.loss
        total_val_loss += loss.item()

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        flat_predictions = predictions.view(-1)
        flat_references = batch["labels"].view(-1)
        flat_attention_mask = batch["labels"].view(-1) < NO_CLASS

        active_predictions = flat_predictions[flat_attention_mask]
        active_references = flat_references[flat_attention_mask]

        all_predictions.extend(active_predictions.cpu().numpy())
        all_references.extend(active_references.cpu().numpy())

    avg_val_loss = total_val_loss / len(eval_dataloader)
    val_losses.append(avg_val_loss)

    val_accuracy = accuracy_score(all_references, all_predictions)
    val_f1 = f1_score(all_references, all_predictions, average="macro")

    val_accuracies.append(val_accuracy)
    val_f1_scores.append(val_f1)

    # Log metrics to TensorBoard
    writer.add_scalar('Loss/Train', avg_train_loss, epoch)
    writer.add_scalar('Loss/Validation', avg_val_loss, epoch)
    writer.add_scalar('Accuracy/Validation', val_accuracy, epoch)
    writer.add_scalar('F1/Validation', val_f1, epoch)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Training Loss: {avg_train_loss:.4f}")
    print(f"Validation Loss: {avg_val_loss:.4f}")
    print(f"Validation Accuracy: {val_accuracy:.4f}")
    print(f"Validation F1 Score: {val_f1:.4f}")

# Save the model
save_dir = "model_and_logs"
model_save_path = os.path.join(save_dir, "model.pth")
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

# Close the TensorBoard writer
writer.close()

Validation Epoch 1/3: 100%|██████████| 782/782 [00:57<00:00, 13.62it/s]


Epoch 1/3
Training Loss: 2.3355
Validation Loss: 2.7463
Validation Accuracy: 0.1691
Validation F1 Score: 0.0858


Training Epoch 2/3:  23%|██▎       | 1093/4688 [01:33<05:07, 11.68it/s]


KeyboardInterrupt: 

In [ ]:
from tqdm.auto import tqdm

progress_bar_train = tqdm(range(num_training_steps), desc="Training Progress")
progress_bar_eval = tqdm(range(num_epochs * len(eval_dataloader)), desc="Evaluation Progress")

for epoch in range(num_epochs):
  model.train()
  for batch in train_dataloader:
      batch = {k: v.to(device) for k, v in batch.items()}
      outputs = model(**batch)
      loss = outputs.loss
      loss.backward()

      optimizer.step()
      lr_scheduler.step()
      optimizer.zero_grad()
      progress_bar_train.update(1)
  
  print("Epoch:", epoch, "Loss:", loss.item())

  model.eval()
  for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    # Logits Size: (batch_size, sequence_length, num_labels)
    logits = outputs.logits

    # Predictions: (batch_size, sequence_length)
    predictions = torch.argmax(logits, dim=-1)


    # Flatten the predictions, references, and attention mask
    flat_predictions = predictions.view(-1)
    flat_references = batch["labels"].view(-1)
    flat_attention_mask = batch["labels"].view(-1) < NO_CLASS

    # Filter out the padding tokens
    active_predictions = flat_predictions[flat_attention_mask]
    active_references = flat_references[flat_attention_mask]

    metric.add_batch(predictions=active_predictions, references=active_references)

    metric_acc.add_batch(predictions=active_predictions, references=active_references)
  
    progress_bar_eval.update(1)
    
  # Calculate the F1 score
  result = metric.compute(predictions=active_predictions, references=active_references, average="macro")
  print("Macro F1 Score:", result['f1'])

  result_acc = metric_acc.compute(predictions=active_predictions, references=active_references)
  print("Accuracy:", result_acc['accuracy'])

  # # You can also calculate the F1 score with micro and weighted averaging
  # result_micro = metric.compute(predictions=active_predictions, references=active_references, average="micro")
  # print("Micro F1 Score:", result_micro['f1'])

  # result_weighted = metric.compute(predictions=active_predictions, references=active_references, average="weighted")
  # print("Weighted F1 Score:", result_weighted['f1'])


Training Progress:  33%|███▎      | 3905/11721 [03:59<08:08, 15.99it/s]

Epoch: 0 Loss: 2.0151631832122803


Training Progress:  33%|███▎      | 3907/11721 [04:16<08:08, 15.99it/s]

Macro F1 Score: 0.19989245780120005


KeyError: 'acc'

## Fine Tuning and CharacterPredictor Head


In [ ]:
import torch.nn as nn
import torch

class CharacterPredictor(nn.Module):
    def __init__(self, embedding_dim, vocab_size):
        super(CharacterPredictor, self).__init__()
        self.fc = nn.Linear(embedding_dim, vocab_size)
    
    def forward(self, x):
        return self.fc(x)

# Example: Assume `vocab_size` is the size of your character vocabulary
vocab_size = len(tokenizer.get_vocab())
embedding_dim = embeddings.shape[-1]
predictor = CharacterPredictor(embedding_dim, vocab_size)

# Predict masked characters
masked_indices = [i for i, char in enumerate(masked_word.split()) if char == '_']
predictions = predictor(embeddings[:, masked_indices, :])

# Convert predictions to characters
predicted_chars = [tokenizer.decode(torch.argmax(pred, dim=-1)) for pred in predictions]
print(predicted_chars)

In [ ]:
import torch.nn as nn
import torch

class CharacterPredictor(nn.Module):
    def __init__(self, embedding_dim, vocab_size):
        super(CharacterPredictor, self).__init__()
        self.fc = nn.Linear(embedding_dim, vocab_size)
    
    def forward(self, x):
        return self.fc(x)

# Example: Assume `vocab_size` is the size of your character vocabulary
vocab_size = len(tokenizer.get_vocab())
embedding_dim = embeddings.shape[-1]
predictor = CharacterPredictor(embedding_dim, vocab_size)

# Predict masked characters
masked_indices = [i for i, char in enumerate(masked_word.split()) if char == '_']
predictions = predictor(embeddings[:, masked_indices, :])

# Convert predictions to characters
predicted_chars = [tokenizer.decode(torch.argmax(pred, dim=-1)) for pred in predictions]
print(predicted_chars)

## Inference

In [ ]:
# Feed batch to CharacterBERT & get the embeddings
embeddings_for_batch, _ = model(batch_ids)
print("Output: ", embeddings_for_batch.size())
embeddings_for_x = embeddings_for_batch[0]
print('These are the embeddings produces by CharacterBERT (last transformer layer)')
for token, embedding in zip(x, embeddings_for_x):
    print(token, embedding.size())

In [ ]:
https://stackoverflow.com/questions/69907682/what-are-differences-between-automodelforsequenceclassification-vs-automodel

https://huggingface.co/learn/nlp-course/chapter3/4?fw=pt

https://aclanthology.org/2020.coling-main.609.pdf

https://github.com/helboukkouri/character-bert?tab=readme-ov-file#how-do-i-pre-train-characterbert